In [1]:
%matplotlib inline
# import sys
# sys.path
# sys.path.append('.')
# print(sys.path)

In [2]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from DA_Dataset import DA_Dataset

In [3]:
#load train
B = 1
data_path = '../../DA_data/train/'
trainset = DA_Dataset(data_path)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=B,
                                          shuffle=True, num_workers=2)

In [4]:
#load test

# data_path = '../../DA_data/test/'
# testset = DA_Dat'../../aset(data_path)
# testloader = torch.utils.data.DataLoader(valset, batch_size=4,
#                                          shuffle=False, num_workers=2)

In [5]:
classes = (1, 2, 3, 4)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [7]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        #layer 1 
        self.conv1  = nn.Conv1d(1, 8, 7)
        self.batch1 = nn.BatchNorm1d(8)
        self.relu1  = nn.ReLU()
        self.drop1  = nn.Dropout()
        self.pool1  = nn.MaxPool1d(2, stride=2)
        
        #layer 2
        self.conv2  = nn.Conv1d(8, 16, 7)
        self.batch2 = nn.BatchNorm1d(16)
        self.relu2  = nn.ReLU()
        self.drop2  = nn.Dropout()
        self.pool2  = nn.MaxPool1d(2, stride=2)
       
        #layer 3 
        self.conv3  = nn.Conv1d(16, 32, 7)
        self.batch3 = nn.BatchNorm1d(32)
        self.relu3  = nn.ReLU()
        
        #fully
        self.fc1 = nn.Linear(177 * 32, 16)
        self.fc2 = nn.Linear(16, 4)
    
    def forward(self, x):
        
        #layer 1 
        x = self.conv1(x)  
        x = self.batch1(x) 
        x = self.relu1(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        #layer 2
        x = self.conv2(x)
        x = self.batch2(x)
        x = self.relu2(x)
        x = self.drop2(x)
        x = self.pool2(x)
       
        #layer 3 
        x = self.conv3(x)
        x = self.batch3(x)
        x = self.relu3(x)
        
        x=x.view(-1, 177 * 32)
        
        #fully
        x = self.fc1(x)
        x = self.fc2(x)

        return x


net = Net()

net.to(device)

Net(
  (conv1): Conv1d(1, 8, kernel_size=(7,), stride=(1,))
  (batch1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (drop1): Dropout(p=0.5)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(8, 16, kernel_size=(7,), stride=(1,))
  (batch2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (drop2): Dropout(p=0.5)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(16, 32, kernel_size=(7,), stride=(1,))
  (batch3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc1): Linear(in_features=5664, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=4, bias=True)
)

In [8]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [9]:
NUM_OF_EPOC = 10
for epoch in range(NUM_OF_EPOC):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
#         print(type(data))
#         print(data)
        inputs, labels = data['x'], data['lable']
        
#         print(np.shape(inputs))
#         print(np.shape(labels))
#         print(type(inputs))
#         print(type(labels))
#         print(inputs)
#         print(labels)
        
        inputs =  inputs.to(device).float()
        labels =  labels.to(device).long()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        print(type(outputs))
        print(type(labels))
        print(outputs.dtype)
        print(labels.dtype)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

<class 'torch.Tensor'>
<class 'torch.Tensor'>
torch.float32
torch.int64


RuntimeError: multi-target not supported at /pytorch/aten/src/THCUNN/generic/ClassNLLCriterion.cu:15

5. Test the network on the test data
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

We have trained the network for 2 passes over the training dataset.
But we need to check if the network has learnt anything at all.

We will check this by predicting the class label that the neural network
outputs, and checking it against the ground-truth. If the prediction is
correct, we add the sample to the list of correct predictions.

Okay, first step. Let us display an image from the test set to get familiar.



In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

Okay, now let us see what the neural network thinks these examples above are:



In [ ]:
outputs = net(images)

The outputs are energies for the 10 classes.
The higher the energy for a class, the more the network
thinks that the image is of the particular class.
So, let's get the index of the highest energy:



In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

The results seem pretty good.

Let us look at how the network performs on the whole dataset.



In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

That looks waaay better than chance, which is 10% accuracy (randomly picking
a class out of 10 classes).
Seems like the network learnt something.

Hmmm, what are the classes that performed well, and the classes that did
not perform well:



In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Okay, so what next?

How do we run these neural networks on the GPU?

Training on GPU
----------------
Just like how you transfer a Tensor onto the GPU, you transfer the neural
net onto the GPU.

Let's first define our device as the first visible cuda device if we have
CUDA available:



In [ ]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net.to(device)

The rest of this section assumes that ``device`` is a CUDA device.

Then these methods will recursively go over all modules and convert their
parameters and buffers to CUDA tensors:

.. code:: python

    net.to(device)


Remember that you will have to send the inputs and targets at every step
to the GPU too:

.. code:: python

        inputs, labels = inputs.to(device), labels.to(device)

Why dont I notice MASSIVE speedup compared to CPU? Because your network
is realllly small.

**Exercise:** Try increasing the width of your network (argument 2 of
the first ``nn.Conv2d``, and argument 1 of the second ``nn.Conv2d`` –
they need to be the same number), see what kind of speedup you get.

**Goals achieved**:

- Understanding PyTorch's Tensor library and neural networks at a high level.
- Train a small neural network to classify images

Training on multiple GPUs
-------------------------
If you want to see even more MASSIVE speedup using all of your GPUs,
please check out :doc:`data_parallel_tutorial`.

Where do I go next?
-------------------

-  :doc:`Train neural nets to play video games </intermediate/reinforcement_q_learning>`
-  `Train a state-of-the-art ResNet network on imagenet`_
-  `Train a face generator using Generative Adversarial Networks`_
-  `Train a word-level language model using Recurrent LSTM networks`_
-  `More examples`_
-  `More tutorials`_
-  `Discuss PyTorch on the Forums`_
-  `Chat with other users on Slack`_


